**code after reducing** -> punctuation, numbers, stopwords, emojis, English words
<!-- Lemmatized -->
**keeping** ->

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import numpy as np
import pandas as pd
import pickle
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import re,json,nltk
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report,accuracy_score,precision_score,recall_score,f1_score
#stopwords_list ='stopwords-bn.txt'


import re
from google.colab import files
import string

In [ ]:
import torch
from transformers import BertTokenizer, BertModel
from sklearn.preprocessing import LabelEncoder
from tqdm import tqdm
import pandas as pd

In [ ]:
data = pd.read_excel('/content/drive/MyDrive/4.2_resources/softcom/data')
print("Total Reviews:",len(data),
      "\nTotal Positive Reviews:",len(data[data.Sentiment == 1]),
      "\nTotal Neutral Reviews:",len(data[data.Sentiment == 2]),
      "\nTotal Negative Reviews:",len(data[data.Sentiment == 0]))

In [ ]:
data.columns

In [ ]:
# print some unprocessed reviews
sample_data = [573, 1209, 864, 297, 1502, 639, 2085, 431, 775, 1926, 1098, 256, 1823, 904, 117, 1985, 623, 1490, 765, 2217]
for i in sample_data:
      print(data.Comment[i],'\n','Sentiment:-- ',data.Tag[i],'\n')

In [ ]:
import re
import string

def clean_text(text):
    # Remove numbers
    text = re.sub(r'\d+', '', str(text))

    # Remove all punctuation using string.punctuation
    # review = re.sub('[^\u0980-\u09FF]',' ',str(review)) #removing unnecessary punctuation
    text = re.sub('[^\u0980-\u09FF]',' ',str(text))  # Keeping Bengali characters and specified punctuation

    # Remove English words
    text = re.sub(r'[A-Za-z]+', '', str(text))

    # Remove emojis
    emoji_pattern = re.compile("["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
        u"\U00002500-\U00002BEF"  # chinese char
        u"\U00002702-\U000027B0"
        u"\U000024C2-\U0001F251"
        u"\U0001f926-\U0001f937"
        u"\U00010000-\U0010ffff"
        u"\u2640-\u2642"
        u"\u2600-\u2B55"
        u"\u200d"
        u"\u23cf"
        u"\u23e9"
        u"\u231a"
        u"\ufe0f"  # dingbats
        u"\u3030"
                      "]+", re.UNICODE)
    text = emoji_pattern.sub(r'', str(text))

    return text

In [ ]:
# Apply the function to the dataframe

data['cleaned'] = data['Comment'].apply(clean_text)

# Print some cleaned reviews from the dataset
sample_data = [573, 1209, 864, 297, 1502, 639, 2085, 431, 775, 1926, 1098, 256, 1823, 904, 117, 1985, 623, 1490, 765, 2217]

for i in sample_data:
    print('Original:\n', data.Comment[i], '\nCleaned:\n', data.cleaned[i], '\n', 'Sentiment:-- ', data.Tag[i], '\n')


In [ ]:
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')

# Load Bengali stopwords
stop_words = set(stopwords.words('bengali'))

# Function to remove stopwords from text
def remove_stopwords(text):
    words = text.split()  # Split the text into words
    filtered_words = [word for word in words if word.lower() not in stop_words]  # Remove stopwords
    return ' '.join(filtered_words)  # Recreate text without stopwords

In [ ]:
print(stop_words)

{'যাওয়া', 'ছাড়াও', 'কমনে', 'যাতে', 'পক্ষে', 'এত', 'জন্যওজে', 'কাউকে', 'দেয়', 'চার', 'হলেই', 'কিন্তু', 'পেয়ে', 'যেন', 'রেখে', 'ই', 'যাদের', 'এটা', 'করার', 'করতে', 'কেন', 'হয়ে', 'মধ্যে', 'দেওয়া', 'যাঁর', 'পি', 'আপনি', 'বক্তব্য', 'তো', 'যার', 'কত', 'নিজেদের', 'প্রভৃতি', 'এতটাই', 'বরং', 'যতটা', 'হবে', 'নাকি', 'পরেই', 'চেয়ে', 'দিতে', 'অন্য', 'কিংবা', 'সেটাই', 'দেওয়া', 'হত', 'বললেন', 'বলা', 'এল', 'ফলে', 'ওখানে', 'পাচ', 'মতো', 'কোনও', 'শুরু', 'অবধি', 'স্পষ্ট', 'হয়নি', 'হইতে', 'থাকবেন', 'হতেই', 'দিয়েছেন', 'নানা', 'তুমি', 'এখানেই', 'সি', 'আগেই', 'আজ', 'তাঁরা', 'করলে', 'ধরা', 'সম্প্রতি', 'জানানো', 'ওদের', 'কাজ', 'কোনো', 'বি', 'হইবে', 'প্রায়', 'তারৈ', 'আছে', 'কেউ', 'করেন', 'তাহা', 'তবে', 'বিনা', 'কয়েক', 'সেটি', 'ফের', 'যখন', 'বলতে', 'প্রযন্ত', 'অবশ্য', 'জনকে', 'নেওয়া', 'সুতরাং', 'হতে', 'জানা', 'এর', 'তাদের', 'দ্বারা', 'গিয়েছে', 'হয়তো', 'রাখা', 'করে', 'ভাবে', 'আগে', 'ওঁদের', 'জানায়', 'এদের', 'দিকে', 'র', 'তারা', 'মাধ্যমে', 'গেল', 'এব', 'করেছিলেন', 'সঙ্গেও', 'এঁরা', 'সামনে', 'আমরা', 'ত

In [ ]:
# Apply the function to the dataframe
data['cleaned_stopwords'] = data['cleaned'].apply(remove_stopwords)

# Print some cleaned reviews from the dataset
sample_data = [573, 1209, 864, 297, 1502, 639, 2085, 431, 775, 1926, 1098, 256, 1823, 904, 117, 1985, 623, 1490, 765, 2217]

for i in sample_data:
    print('cleaned:\n', data.cleaned[i], '\ncleaned_stopwords:\n', data.cleaned_stopwords[i], '\n', 'Sentiment:-- ', data.Tag[i], '\n')


In [ ]:
!pip install BnLemma

In [ ]:
import BnLemma as lm

# Function to remove stopwords from text
def ben_Lemmatizer(text):
    bl = lm.Lemmatizer()
    return bl.lemma(text)


data['Lemmatized'] = data['cleaned'].apply(ben_Lemmatizer)



# Example print after removing stopwords
for i in sample_data:
      print('Original:\n',data.cleaned[i],'\nCleaned:\n',data.Lemmatized[i],'\n','Sentiment:-- ',data.Sentiment[i],'\n')

In [ ]:
#should we remove low length data...?

In [ ]:
data.columns

In [ ]:
#pip install transformers

In [ ]:
import pandas as pd

# Assuming 'data' is your DataFrame
selected_columns = ['cleaned_stopwords', 'Tag']
selected_data = data[selected_columns]

# Save the selected columns to a CSV file
selected_data.to_csv('/content/drive/MyDrive/4.2_resources/softcom/data/project_selected_dataA.csv', index=False)

In [ ]:
cleaned_data = pd.read_csv('/content/drive/MyDrive/4.2_resources/softcom/data/project_selected_dataA.csv')

In [ ]:
print(cleaned_data.columns)
print("Total Reviews:",len(cleaned_data))

In [ ]:
cleaned_data.columns

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report
from transformers import BertTokenizer, BertModel
import torch


feature_columns = ['cleaned_stopwords']

# X = cleaned_data[feature_columns].iloc[:1000]
# y = cleaned_data['Tag'].iloc[:1000]

X = cleaned_data[feature_columns]
y = cleaned_data['Tag']

label_mapping = {'Negative': 0, 'Positive': 1, 'Neutral': 2}
y_encoded = y.map(label_mapping)

tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased')
model = BertModel.from_pretrained('bert-base-multilingual-cased')

batch_size = 2
tokenized_data = []
for i in range(0, len(X), batch_size):
    batch_texts = X['cleaned_stopwords'].iloc[i:i+batch_size].tolist()
    batch_encoding = tokenizer.batch_encode_plus(
        batch_texts,
        add_special_tokens=True,
        padding='max_length',
        truncation=True,
        max_length=128,
        return_tensors='pt'
    )
    tokenized_data.append(batch_encoding)

input_ids = torch.cat([d['input_ids'] for d in tokenized_data], dim=0)
attention_mask = torch.cat([d['attention_mask'] for d in tokenized_data], dim=0)

with torch.no_grad():
    model.eval()
    outputs = model(input_ids, attention_mask=attention_mask)

bert_embeddings = outputs.last_hidden_state[:, 0, :]


In [ ]:
len(bert_embeddings)

In [ ]:
torch.save(bert_embeddings, '/content/drive/MyDrive/4.2_resources/softcom/data/bert_embeddingsA.pt')

In [ ]:
loaded_embeddings = torch.load('/content/drive/MyDrive/4.2_resources/softcom/data/bert_embeddingsA.pt')

In [ ]:
import pickle

In [ ]:
import pickle

save_dict = {
    'bert_embeddings': bert_embeddings,
    'y_encoded': y_encoded
}

with open('/content/drive/MyDrive/4.2_resources/softcom/data/bert_and_y_encodedA.pkl', 'wb') as file:
    pickle.dump(save_dict, file)


In [ ]:
with open('/content/drive/MyDrive/4.2_resources/softcom/data/bert_and_y_encodedA.pkl', 'rb') as file:
    loaded_dict = pickle.load(file)

loaded_bert_embeddings = loaded_dict['bert_embeddings']
loaded_y_encoded = loaded_dict['y_encoded']


In [ ]:
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, classification_report
from sklearn.naive_bayes import MultinomialNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.metrics import accuracy_score, classification_report

def batch_train(model, X_train_batches, y_train_batches, X_test, y_test):
    for X_batch, y_batch in zip(X_train_batches, y_train_batches):
        model.fit(X_batch, y_batch)

    y_pred = model.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    classification_rep = classification_report(y_test, y_pred)

    return accuracy, classification_rep

# Assuming loaded_bert_embeddings and loaded_y_encoded are defined
X_train, X_test, y_train, y_test = train_test_split(loaded_bert_embeddings, loaded_y_encoded, test_size=0.2, random_state=42)

batch_size = 100

X_train_batches = [X_train[i:i+batch_size] for i in range(0, len(X_train), batch_size)]
y_train_batches = [y_train[i:i+batch_size] for i in range(0, len(y_train), batch_size)]

# K-Nearest Neighbors (KNN) Classifier
knn_classifier = KNeighborsClassifier(n_neighbors=5)
knn_accuracy, knn_classification_rep = batch_train(knn_classifier, X_train_batches, y_train_batches, X_test, y_test)

print("K-Nearest Neighbors (KNN) Classifier:")
print(f"Accuracy: {knn_accuracy}")
print("Classification Report:\n", knn_classification_rep)

# Decision Tree (DT) Classifier
dt_classifier = DecisionTreeClassifier(random_state=42)
dt_accuracy, dt_classification_rep = batch_train(dt_classifier, X_train_batches, y_train_batches, X_test, y_test)

print("\nDecision Tree (DT) Classifier:")
print(f"Accuracy: {dt_accuracy}")
print("Classification Report:\n", dt_classification_rep)

# Linear Support Vector Machine (SVM) Classifier
linear_svm_classifier = SVC(kernel='linear', C=1.0)
linear_svm_accuracy, linear_svm_classification_rep = batch_train(linear_svm_classifier, X_train_batches, y_train_batches, X_test, y_test)

print("\nLinear Support Vector Machine (SVM) Classifier:")
print(f"Accuracy: {linear_svm_accuracy}")
print("Classification Report:\n", linear_svm_classification_rep)

# Random Forest Classifier
rf_classifier = RandomForestClassifier(n_estimators=100, random_state=42)
rf_accuracy, rf_classification_rep = batch_train(rf_classifier, X_train_batches, y_train_batches, X_test, y_test)

print("\nRandom Forest Classifier:")
print(f"Accuracy: {rf_accuracy}")
print("Classification Report:\n", rf_classification_rep)

# Gradient Boosting Classifier
gb_classifier = GradientBoostingClassifier(n_estimators=100, random_state=42)
gb_accuracy, gb_classification_rep = batch_train(gb_classifier, X_train_batches, y_train_batches, X_test, y_test)

print("\nGradient Boosting Classifier:")
print(f"Accuracy: {gb_accuracy}")
print("Classification Report:\n", gb_classification_rep)

In [ ]:
# # code to appy epoch
# from sklearn.model_selection import train_test_split
# from sklearn.neighbors import KNeighborsClassifier
# from sklearn.tree import DecisionTreeClassifier
# from sklearn.svm import SVC
# from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
# from sklearn.metrics import accuracy_score, classification_report

# def batch_train(model, X_train_batches, y_train_batches, X_test, y_test, epochs=1):
#     for epoch in range(epochs):
#         for X_batch, y_batch in zip(X_train_batches, y_train_batches):
#             model.fit(X_batch, y_batch)

#         y_pred = model.predict(X_test)
#         accuracy = accuracy_score(y_test, y_pred)
#         classification_rep = classification_report(y_test, y_pred)

#         print(f"Epoch {epoch + 1} - Accuracy: {accuracy}")
#         print("Classification Report:\n", classification_rep)

# # Assuming loaded_bert_embeddings and loaded_y_encoded are defined
# X_train, X_test, y_train, y_test = train_test_split(loaded_bert_embeddings, loaded_y_encoded, test_size=0.2, random_state=42)

# batch_size = 100
# epochs = 3  # You can adjust the number of epochs

# X_train_batches = [X_train[i:i+batch_size] for i in range(0, len(X_train), batch_size)]
# y_train_batches = [y_train[i:i+batch_size] for i in range(0, len(y_train), batch_size)]

# # K-Nearest Neighbors (KNN) Classifier
# knn_classifier = KNeighborsClassifier(n_neighbors=5)
# batch_train(knn_classifier, X_train_batches, y_train_batches, X_test, y_test, epochs=epochs)

# # ... Repeat for other classifiers


In [ ]:
print(len(loaded_bert_embeddings))
print(len(loaded_y_encoded))

In [ ]:
loaded_bert_embeddings

In [ ]:
loaded_y_encoded

In [ ]:
# end end end  end end end  end end end  end end end  end end end  end end end  end end end  end end end
# end end end  end end end  end end end  end end end  end end end  end end end  end end end  end end end
# end end end  end end end  end end end  end end end  end end end  end end end  end end end  end end end
# end end end  end end end  end end end  end end end  end end end  end end end  end end end  end end end